In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from utils import *
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## Load the data

In [2]:
# datapath = '/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/UCSD/DSC/DSC180/ClimateBench - Plus/ClimateBench-Plus/DKL Gaussian Process/data/processed_data/'
datapath = 'G://My Drive//UCSD//DSC//DSC180//ClimateBench - Plus//ClimateBench-Plus//DKL Gaussian Process//data//processed_data//'
simulations = ['ssp126', 'ssp370', 'ssp585', 'hist-GHG', 'hist-aer']

In [3]:
X_train = []
Y_train = []

for i, simu in enumerate(simulations):
    input_name = 'inputs_' + simu + '.nc'
    output_name = 'outputs_' + simu + '.nc'
    # Just load hist data in these cases 'hist-GHG' and 'hist-aer'
    if 'hist' in simu:
        # load inputs 
        input_xr = xr.open_dataset(datapath + input_name)
            
        # load outputs                                                             
        output_xr = xr.open_dataset(datapath + output_name).mean(dim='member')
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400, "pr90": output_xr.pr90 * 86400})\
                             .rename({'lon':'longitude', 'lat': 'latitude'})\
                             .transpose('time','latitude', 'longitude').drop(['quantile'])
    
    # Concatenate with historical data in the case of scenario 'ssp126', 'ssp370' and 'ssp585'
    else:
        # load inputs 
        input_xr = xr.open_mfdataset([datapath + 'inputs_historical.nc', datapath + input_name]).compute()
            
        # load outputs                                                             
        output_xr = xr.concat([xr.open_dataset(datapath + 'outputs_historical.nc').mean(dim='member'),
                               xr.open_dataset(datapath + output_name).mean(dim='member')],
                               dim='time').compute()
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400,"pr90": output_xr.pr90 * 86400})\
                             .rename({'lon':'longitude', 'lat': 'latitude'})\
                             .transpose('time','latitude', 'longitude').drop(['quantile'])

    print(input_xr.dims, simu)

    # Append to list 
    X_train.append(input_xr)
    Y_train.append(output_xr)

Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp126
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp370
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp585
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-GHG
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-aer


## Normalize the data


In [4]:
# Compute mean/std of each variable for the whole dataset
meanstd_inputs = {}
len_historical = 165

for var in ['CO2', 'CH4', 'SO2', 'BC']:
    # To not take the historical data into account several time we have to slice the scenario datasets
    # and only keep the historical data once (in the first ssp index 0 in the simus list)
    array = np.concatenate([X_train[i][var].data for i in [0, 3, 4]] + 
                           [X_train[i][var].sel(time=slice(len_historical, None)).data for i in range(1, 3)])
    print((array.mean(), array.std()))
    meanstd_inputs[var] = (array.mean(), array.std())

(1074.172303244536, 1755.690699230666)
(0.1927369743762821, 0.18457590641432994)
(2.5623359997066755e-12, 2.250114566783271e-11)
(1.4947905009818064e-13, 1.0313342554838387e-12)


In [5]:
# normalize input data 
X_train_norm = [] 
for i, train_xr in enumerate(X_train): 
    for var in ['CO2', 'CH4', 'SO2', 'BC']: 
        var_dims = train_xr[var].dims
        train_xr=train_xr.assign({var: (var_dims, normalize(train_xr[var].data, var, meanstd_inputs))}) 
    X_train_norm.append(train_xr)

In [21]:
var_to_predict =  'tas'
# skip_historical set to (i < 2) because of the order of the scenario and historical runs in the X_train and Y_train lists.
# In details: ssp126 0, ssp370 1 = skip historical part of the data, ssp585 2, hist-GHG 3 and hist-aer 4 = keep the whole sequence
X_train_all = np.concatenate([input_for_training(X_train_norm[i], skip_historical=(i<2), len_historical=len_historical) for i in range(len(simulations))], axis = 0)
Y_train_all = np.concatenate([output_for_training(Y_train[i], var_to_predict, skip_historical=(i<2), len_historical=len_historical) for i in range(len(simulations))], axis=0)
# add a dimension to the output data
Y_train_all = Y_train_all[..., np.newaxis]
print(X_train_all.shape)
print(Y_train_all.shape)

(726, 96, 144, 4)
(726, 96, 144, 1)


## Model

Neural Process Arch
- Encoder --> Takes context points x_i, y_i to r_i (deterministic path) and s_i (latent path)
- **r_c, s_c** from context points representations aggregated by mean
- Decoder --> Takes r_c, s_c, target points x_i to predict y_i

Pytorch implementation --> https://github.com/EmilienDupont/neural-processes/blob/master/neural_process.py

In [15]:
import torch
from torch import nn
from torch.nn import functional as F
from model import TimeDistributed
from np import Encoder

### Encoder
- Input: x_i, y_i (context points)
    - x_i: (726, 10, 96, 144, 4)
    - y_i: (726, 1, 96, 144)
- Output: r_i, s_i

Plan to use LSTM/CNN network to encode the context points into the representations

#### Changes after meeting
- Reduce shape to (726, 96, 144, 4) and (726, 96, 144, 1) respectively since time covariance should not matter too much in this case,
- Remove LSTM layer, since time slider shouldn't matter and would make the problem easier. 
- If doesn't work, we can flatten the latlng layer too

In [84]:
class Encoder(nn.Module):
    """Maps an (x_i, y_i) pair to a representation r_i.

    Parameters
    ----------
    x_dim : int
        Dimension of x values.

    y_dim : int
        Dimension of y values.

    h_dim : int
        Dimension of hidden layer.

    r_dim : int
        Dimension of output representation r.
    """
    def __init__(self, x_dim, y_dim, h_dim, r_dim):
        super(Encoder, self).__init__()

        self.x_dim = x_dim
        self.y_dim = y_dim
        self.h_dim = h_dim
        self.r_dim = r_dim


        self.conv2d = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=(3, 3), padding=1)
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.relu = nn.ReLU(inplace=True)
        self.hidden = nn.Linear(1*96*144, r_dim)

    def forward(self, context_points):
        """
        x : torch.Tensor
            Shape (batch_size, x_dim)

        y : torch.Tensor
            Shape (batch_size, y_dim)
        """
        context_points = self.relu(self.conv2d(context_points))
        print('2d', context_points.shape)
        context_points = self.avg_pool(context_points)
        print('avgpool', context_points.shape)
        context_points = self.global_avg_pool(context_points)
        print('globalavgpool', context_points.shape)
        context_points = self.relu(context_points)
        print(context_points.shape)
        context_points = self.hidden(context_points.view(-1, 1*96*144))

        return context_points
        


In [85]:
X_train_all = torch.tensor(X_train_all).float()
Y_train_all = torch.tensor(Y_train_all).float()

context_points = torch.concatenate([X_train_all, Y_train_all], dim=-1)

en = Encoder(
    1,  # number of input features
    64,  # number of output features
    512,  # number of hidden layer dim
    512 # number of r dim
)

In [86]:
en(context_points)

2d torch.Size([726, 96, 144, 5])
avgpool torch.Size([726, 96, 72, 2])
globalavgpool torch.Size([726, 96, 1, 1])
torch.Size([726, 96, 1, 1])


RuntimeError: shape '[-1, 13824]' is invalid for input of size 69696